In [ ]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import pathlib
import seaborn as sns
import pandas as pd
sns.set_context('talk')
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path_list = [
    # "/storage/vbutoi/scratch/ESE/training/older_runs/2024/10_October_2024/10_23_24_WMH_3D_Models_wHeavyAug",
    # "/storage/vbutoi/scratch/ESE/training/older_runs/2024/10_October_2024/10_14_24_ISLES_3D_CrossEntropy_HeavyAug",
    "/storage/vbutoi/scratch/ESE/training/older_runs/2024/10_October_2024/10_15_24_ISLES_3D_CrossEntropy_HeavyAug_1e-4",
]

df_list_1 = []
for path in path_list:
    try:
        dfc = rs.load_configs(
            path,
            properties=False,
        )
        df = rs.load_metrics(dfc)
        df_list_1.append(df)
    except Exception as e:
        print(e)

df = pd.concat(df_list_1)

In [ ]:
def exp_name(path):
    exp_root = str(path).split("/")[-2]
    return exp_root

df.augment(exp_name)

In [ ]:
for ikey in df.keys():
    print(ikey)

In [ ]:
df.unique_per_col()

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss',
    # hue='loss_func',
    hue='seed',
    col='phase',
    kind='line',
    height=12,
    facet_kws=dict(sharey=False),
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.12)
# Set the y axis to be between 0 and 1
# g.set(ylim=(0, 1000))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='dice_score',
    # hue='loss_func',
    hue='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.12)